# Example of how to use Deep Learning for text classification

Click [Text Classification Example](https://colab.research.google.com/github/magic-lantern/coprh-nlp-2021/blob/main/text_classification_colab.ipynb) to open this notebook in Google Colab - no local setup required, results saved to your Google Drive.

In Google Colab, make sure you change the Runtime to a GPU instance - that will result vastly improved runtimes for deep learning steps. On CPU based hardware some steps take 40 minutes versus 1 minute per training epoch on systems with a GPU. Steps to update runtime:

    Runtime > Change runtime type > Select 'GPU' in Hardware accelerator box

Click 'Save' to switch from Default runtime to GPU accelerated runtime.

Some useful links:

* https://muellerzr.github.io/fastblog/2020/08/21/intermediate.html#A-Text-Example

In [ ]:
# this cell makes sure Google Colab version has latest software dependencies
!pip install -Uqq fastbook fastai

In [ ]:
import fastbook
import gc

In [ ]:
# this cell will setup a path that allows you to load and save files to your google drive
# you will be prompted to log in with a Google enabled account.
fastbook.setup_book()

In [ ]:
from fastai.text.all import *

In [ ]:
Path.cwd()

In [ ]:
data_path = fastbook.gdrive / Path('data')
data_file = data_path / 'mtsamples.csv'
sequence_length = 512

In [ ]:
data_file

In [ ]:
if data_file.is_file():
    print('Already downloaded')
else:
    print('downloading data file')
    download_data(
        url='https://github.com/socd06/medical-nlp/raw/master/data/mtsamples.csv',
        fname = data_file
    )

In [ ]:
mtsamples = pd.read_csv(str(data_path) + '/mtsamples.csv')
mtsamples.head()

A note on data cleanup. In the best case, a machine learning model can only be as good as the data provided. If data is of poor quality or inconsistent, it will be difficult to get good accuracy with your model.

Let's review the really short transcription notes.

In [ ]:
word_len = mtsamples.transcription.str.split(' ').str.len()

In [ ]:
mtsamples['num_words'] =  mtsamples.transcription.str.split(' ').str.len()

In [ ]:
word_len.median()

In [ ]:
word_len.min()

In [ ]:
print(f"There are {len(mtsamples[mtsamples['num_words'] < 5])} records with short notes.")
mtsamples[mtsamples['num_words'] < 5].head()

In [ ]:
# drop transcripts with fewer than 5 words - they are not good notes
mtsamples = mtsamples[mtsamples['num_words'] >= 5]

In [ ]:
mtsamples.medical_specialty.value_counts()

This is one way to load the data for training - it's more simple, but you have less control.

```python
TextDataLoaders.from_df(mtsamples,
                        valid_pct=0.2,
                        seed=42,
                        text_col='transcription',
                        label_col='medical_specialty',
                        seq_len=72)
```

In [ ]:
dls_lm = DataBlock(
    blocks=TextBlock.from_df('transcription', is_lm=True),
    get_x=ColReader('text'),
    splitter=TrainTestSplitter(test_size=0.2,
                               random_state=42)
)

In [ ]:
# if changes in the above were made (such as batch size), it may be beneficial to purge GPU memory
gc.collect()
torch.cuda.empty_cache()
gc.collect()
torch.cuda.empty_cache()

In [ ]:
# This cell takes 1 - 2 minutes
# when using seq_len of 256, bs of 128 fits in GPU memory, bs 256 does not fit
# when using seq_len of 512, bs of 64 fits in GPU memory, bs 128 does not fit
dls_lm = dls_lm.dataloaders(mtsamples, bs=64, seq_len=sequence_length)

In [ ]:
dls_lm.show_batch(max_n=2)

In [ ]:
lm_learn = language_model_learner(
    dls_lm,
    AWD_LSTM,
    model_dir=fastbook.gdrive / Path('data'),
    drop_mult=0.3,
    metrics=[accuracy, Perplexity()]).to_fp16()

In [ ]:
# on macbook this takes 40 minues
# on GPU machine this takes about 1 minute
lm_learn.fit_one_cycle(1, 2e-2)

In [ ]:
lm_learn.save('1stepoch')
# can load already saved trained model with
# learn.load('1stepoch')

In [ ]:
# if changes in the above were made (such as batch size), it may be beneficial to purge GPU memory
# gc.collect()
# torch.cuda.empty_cache()
# gc.collect()
# torch.cuda.empty_cache()

In [ ]:
lm_learn.unfreeze()
lm_learn.fit_one_cycle(20, 2e-3)

In [ ]:
lm_learn.save_encoder('finetuned')
# can load with
# lm_learn.load('finetuned')

## Now build a text classifier

Now that we have finetuned a language model based on our actual text let's build a classifier to identify the medical specialty of the transcription notes. As shown previously in this dataset, some specialties are more represented than others. Split data such that training and testing data sets are balanced based on specialty.

In [ ]:
# this cell takes about 2 minutes to run
dls_clas = DataBlock(
    blocks=(TextBlock.from_df('transcription', vocab=dls_lm.vocab), CategoryBlock),
    get_x=ColReader('text'),
    get_y=ColReader('medical_specialty'),
    splitter=TrainTestSplitter(test_size=0.2,
                               random_state=42,
                               stratify=mtsamples.medical_specialty)
).dataloaders(mtsamples, bs=128, seq_len=sequence_length)

In [ ]:
dls_clas.show_batch(max_n=2)

In [ ]:
learn = text_classifier_learner(
    dls_clas,
    AWD_LSTM,
    drop_mult=0.3,
    metrics=accuracy,
    model_dir=fastbook.gdrive / Path('data')).to_fp16()

In [ ]:
# load the encoder used for the language model - must be the same to build off language model.
learn = learn.load_encoder('finetuned')

In [ ]:
# attempt to find a decent learning rate for this dataset
learn.lr_find()

In [ ]:
learn.fit_one_cycle(1, 3e-1)

In [ ]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(1e-1/(2.6**4),1e-1))

In [ ]:
learn.freeze_to(-3)
learn.fit_one_cycle(1, slice(5e-2/(2.6**4),5e-2))

In [ ]:
# this takes about 30 seconds per epoch, or about 300 seconds total
learn.unfreeze()
learn.fit_one_cycle(50, slice(1e-2/(2.6**4),1e-2))
# after this: learn.fit_one_cycle(10, slice(1e-3/(2.6**4),1e-3))
# starts at 0.33 and got to 0.26 - accuracy is decreasing with training

In [ ]:
learn.show_results()

In [ ]:
interp = Interpretation.from_learner(learn)

In [ ]:
interp.plot_top_losses(9, figsize=(15,10))